In [ ]:
import os 
kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
spark = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"

os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{} pyspark-shell".format(kafka, spark))

from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("spark-job").master("spark://bacener:7077").getOrCreate()

In [ ]:
KAFKA_BROKER='localhost:9092'
KAFKA_TOPIC='item-shopee'
kafkaMessages = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()

In [ ]:
message = kafkaMessages.selectExpr("CAST(value AS STRING)")

In [ ]:
rawQuery = message \
        .writeStream \
        .queryName("dtraw")\
        .format("memory")\
        .start()

In [ ]:
spark.sql("SELECT * FROM dtraw").show()

In [ ]:
fileStream = message.writeStream \
                      .trigger(processingTime='60 seconds')\
                      .queryName("Persist the processed data") \
                      .outputMode("append") \
                      .format("parquet") \
                      .option("path", "hdfs://0.0.0.0:9000/user/tuannha/data") \
                      .option("checkpointLocation", "hdfs://0.0.0.0:9000/user/tuannha/checkpoints") \
                      .start() \
                      .awaitTermination()